In [1]:
import os
import sys
import pickle

current_dir =  os.getcwd()
#parent_dir = os.path.dirname(current_dir)
#sys.path.insert(0, parent_dir)

grandparent_dir = os.path.dirname(os.path.dirname(current_dir))
sys.path.insert(0, grandparent_dir)


import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from cities.utils.cleaning_utils import standardize_and_scale

import matplotlib.pyplot as plt


from cities.utils.cleaning_utils import find_repo_root
root = find_repo_root()


from cities.utils.data_grabber import DataGrabber
data = DataGrabber()
data.get_features_wide(["gdp", "population", "transport",
            "spending_transportation", "spending_commerce", "spending_HHS"])
gdp = data.wide['gdp']
# population = data.wide['population']
# trnasport = data.wide['transport']
# spending_transportation = data.wide['spending_transportation']
# spending_commerce = data.wide['spending_commerce']

def tableInfo(tableName):

    print(tableName.head())
    print(tableName.dtypes)
    print(f'Number of rows: {tableName.shape[0]}')
    print(f'Unique FIPS numbers {tableName['GeoFIPS'].nunique()}')

In [3]:
population_urban.set_index('Label (Grouping)', inplace=True)

# Transpose the DataFrame to swap rows and columns
transposed_df = population_urban.transpose()

# Reset index for a cleaner format
transposed_df.reset_index(inplace=True)

df_population_urban = transposed_df.copy()
df_population_urban.head()

Label (Grouping),index,Total:,Urban,Rural,Not defined for this file
0,"Aberdeen, SD Micro Area","42,287","27,982","14,305",0
1,"Aberdeen, WA Micro Area","75,636","46,051","29,585",0
2,"Abilene, TX Metro Area","176,579","118,138","58,441",0
3,"Ada, OK Micro Area","38,065","17,264","20,801",0
4,"Adrian, MI Micro Area","99,423","42,890","56,533",0


In [4]:
import pandas as pd
filtered_df = pd.DataFrame(df_population_urban[df_population_urban['index'].str.endswith('Metro Area')])


filtered_df = filtered_df.rename(columns={'index': 'MetroName'})

filtered_df.columns = filtered_df.columns.str.replace('Total:', 'total_pop')
filtered_df.columns = filtered_df.columns.str.replace('Urban', 'urban_pop')
filtered_df.columns = filtered_df.columns.str.replace('Rural', 'rural_pop')


filtered_df = filtered_df.iloc[:, :-1].reset_index(drop=True)

population_urban = filtered_df.copy()

population_urban.head()


Label (Grouping),MetroName,total_pop,urban_pop,rural_pop
0,"Abilene, TX Metro Area","176,579","118,138","58,441"
1,"Aguadilla-Isabela, PR Metro Area","310,160","271,123","39,037"
2,"Akron, OH Metro Area","702,219","620,306","81,913"
3,"Albany, GA Metro Area","148,922","92,106","56,816"
4,"Albany-Lebanon, OR Metro Area","128,610","84,567","44,043"


In [6]:
housing_urban.set_index('Label (Grouping)', inplace=True)

# Transpose the DataFrame to swap rows and columns
transposed_df = housing_urban.transpose()

# Reset index for a cleaner format
transposed_df.reset_index(inplace=True)

housing_urban = transposed_df.copy()
housing_urban.head()

Label (Grouping),index,Total:,Urban,Rural,Not defined for this file
0,"Aberdeen, SD Micro Area","19,825","13,246","6,579",0
1,"Aberdeen, WA Micro Area","36,058","22,530","13,528",0
2,"Abilene, TX Metro Area","74,341","50,514","23,827",0
3,"Ada, OK Micro Area","17,406","8,654","8,752",0
4,"Adrian, MI Micro Area","43,577","17,640","25,937",0


In [ ]:
filtered_df = pd.DataFrame(housing_urban[housing_urban['index'].str.endswith('Metro Area')])


filtered_df = filtered_df.rename(columns={'index': 'MetroName'})

filtered_df.columns = filtered_df.columns.str.replace('Total:', 'total_housing')
filtered_df.columns = filtered_df.columns.str.replace('Urban', 'urban_housing')
filtered_df.columns = filtered_df.columns.str.replace('Rural', 'rural_housing')


filtered_df = filtered_df.iloc[:, :-1].reset_index(drop=True)

housing_urban = filtered_df.copy()

housing_urban.head()

In [30]:
# housing_urban
# population_urban

metrolist = pd.read_csv(f"{root}/data/raw/metrolist.csv")

merged_df = housing_urban.merge(population_urban, on='MetroName')

merged_df['MetroName'] = merged_df['MetroName'].str.replace('Metro Area', '(MA)')


df1_subset = metrolist[['GeoFIPS', 'GeoName']].drop_duplicates()

merged_df = pd.merge(merged_df, df1_subset, left_on=['MetroName'], right_on=['GeoName'], how='left')

merged_df = merged_df.drop(columns=['GeoName'])
merged_df.dropna(inplace=True)

merged_df.columns = merged_df.columns.str.strip()
ordered_columns = ['GeoFIPS', 'MetroName', 'total_housing', 'urban_housing', 'rural_housing', 'total_pop', 'urban_pop', 'rural_pop']
ordered_df = merged_df[ordered_columns]

ordered_df = ordered_df.rename(columns={'MetroName': 'GeoName'})

numeric_columns = ['total_housing', 'urban_housing', 'rural_housing', 'total_pop', 'urban_pop', 'rural_pop']
ordered_df[numeric_columns] = ordered_df[numeric_columns].replace({',': ''}, regex=True).astype(float)

ordered_df['GeoFIPS'] = ordered_df['GeoFIPS'].astype(np.int64)



ordered_df['rural_pop_prct'] = (ordered_df['rural_pop'] / ordered_df['total_pop']) 
ordered_df['rural_housing_prct'] = (ordered_df['rural_housing'] / ordered_df['total_housing']) 

ordered_df.drop(['total_pop', 'total_housing'], axis=1, inplace=True)

ordered_df.reset_index(drop=True, inplace=True)

ordered_df




Label (Grouping),MetroName,total_housing,urban_housing,rural_housing,total_pop,urban_pop,rural_pop
0,"Abilene, TX Metro Area","74,341","50,514","23,827","176,579","118,138","58,441"
1,"Aguadilla-Isabela, PR Metro Area","150,383","132,532","17,851","310,160","271,123","39,037"
2,"Akron, OH Metro Area","317,145","282,871","34,274","702,219","620,306","81,913"
3,"Albany, GA Metro Area","66,756","42,590","24,166","148,922","92,106","56,816"
4,"Albany-Lebanon, OR Metro Area","51,921","34,615","17,306","128,610","84,567","44,043"
...,...,...,...,...,...,...,...
387,"Yauco, PR Metro Area","42,322","34,306","8,016","86,142","69,279","16,863"
388,"York-Hanover, PA Metro Area","187,187","134,538","52,649","456,438","326,551","129,887"
389,"Youngstown-Warren-Boardman, OH-PA Metro Area","253,211","190,364","62,847","541,243","397,536","143,707"
390,"Yuba City, CA Metro Area","63,957","47,984","15,973","181,208","142,123","39,085"


In [31]:


merged_df['MetroName'] = merged_df['MetroName'].str.replace('Metro Area', '(MA)')


df1_subset = metrolist[['GeoFIPS', 'GeoName']].drop_duplicates()

merged_df = pd.merge(merged_df, df1_subset, left_on=['MetroName'], right_on=['GeoName'], how='left')

merged_df = merged_df.drop(columns=['GeoName'])
merged_df.dropna(inplace=True)

merged_df.columns = merged_df.columns.str.strip()
ordered_columns = ['GeoFIPS', 'MetroName', 'total_housing', 'urban_housing', 'rural_housing', 'total_pop', 'urban_pop', 'rural_pop']
ordered_df = merged_df[ordered_columns]

ordered_df = ordered_df.rename(columns={'MetroName': 'GeoName'})

numeric_columns = ['total_housing', 'urban_housing', 'rural_housing', 'total_pop', 'urban_pop', 'rural_pop']
ordered_df[numeric_columns] = ordered_df[numeric_columns].replace({',': ''}, regex=True).astype(float)

ordered_df['GeoFIPS'] = ordered_df['GeoFIPS'].astype(np.int64)



ordered_df['rural_pop_prct'] = (ordered_df['rural_pop'] / ordered_df['total_pop']) 
ordered_df['rural_housing_prct'] = (ordered_df['rural_housing'] / ordered_df['total_housing']) 

ordered_df.drop(['total_pop', 'total_housing'], axis=1, inplace=True)

ordered_df.reset_index(drop=True, inplace=True)

ordered_df



,GeoFIPS,GeoName,CountyFIPS,CountyName
0,10180,"Abilene, TX (MA)",48059,"Callahan, TX"
1,10180,"Abilene, TX (MA)",48253,"Jones, TX"
2,10180,"Abilene, TX (MA)",48441,"Taylor, TX"
3,10420,"Akron, OH (MA)",39133,"Portage, OH"
4,10420,"Akron, OH (MA)",39153,"Summit, OH"
...,...,...,...,...
1155,49660,"Youngstown-Warren-Boardman, OH-PA (MA)",39155,"Trumbull, OH"
1156,49660,"Youngstown-Warren-Boardman, OH-PA (MA)",42085,"Mercer, PA"
1157,49700,"Yuba City, CA (MA)",6101,"Sutter, CA"
1158,49700,"Yuba City, CA (MA)",6115,"Yuba, CA"


Label (Grouping),MetroName,total_housing,urban_housing,rural_housing,total_pop,urban_pop,rural_pop
0,"Abilene, TX (MA)","74,341","50,514","23,827","176,579","118,138","58,441"
1,"Aguadilla-Isabela, PR (MA)","150,383","132,532","17,851","310,160","271,123","39,037"
2,"Akron, OH (MA)","317,145","282,871","34,274","702,219","620,306","81,913"
3,"Albany, GA (MA)","66,756","42,590","24,166","148,922","92,106","56,816"
4,"Albany-Lebanon, OR (MA)","51,921","34,615","17,306","128,610","84,567","44,043"
...,...,...,...,...,...,...,...
387,"Yauco, PR (MA)","42,322","34,306","8,016","86,142","69,279","16,863"
388,"York-Hanover, PA (MA)","187,187","134,538","52,649","456,438","326,551","129,887"
389,"Youngstown-Warren-Boardman, OH-PA (MA)","253,211","190,364","62,847","541,243","397,536","143,707"
390,"Yuba City, CA (MA)","63,957","47,984","15,973","181,208","142,123","39,085"


In [33]:
df1_subset = metrolist[['GeoFIPS', 'GeoName']].drop_duplicates()

merged_df = pd.merge(merged_df, df1_subset, left_on=['MetroName'], right_on=['GeoName'], how='left')

merged_df = merged_df.drop(columns=['GeoName'])
merged_df.dropna(inplace=True)

merged_df.columns = merged_df.columns.str.strip()
ordered_columns = ['GeoFIPS', 'MetroName', 'total_housing', 'urban_housing', 'rural_housing', 'total_pop', 'urban_pop', 'rural_pop']
ordered_df = merged_df[ordered_columns]

ordered_df = ordered_df.rename(columns={'MetroName': 'GeoName'})

numeric_columns = ['total_housing', 'urban_housing', 'rural_housing', 'total_pop', 'urban_pop', 'rural_pop']
ordered_df[numeric_columns] = ordered_df[numeric_columns].replace({',': ''}, regex=True).astype(float)

ordered_df['GeoFIPS'] = ordered_df['GeoFIPS'].astype(np.int64)



ordered_df['rural_pop_prct'] = (ordered_df['rural_pop'] / ordered_df['total_pop']) 
ordered_df['rural_housing_prct'] = (ordered_df['rural_housing'] / ordered_df['total_housing']) 

ordered_df.drop(['total_pop', 'total_housing'], axis=1, inplace=True)

ordered_df.reset_index(drop=True, inplace=True)

ordered_df


,MetroName,total_housing,urban_housing,rural_housing,total_pop,urban_pop,rural_pop,GeoFIPS
0,"Abilene, TX (MA)","74,341","50,514","23,827","176,579","118,138","58,441",10180.0
2,"Akron, OH (MA)","317,145","282,871","34,274","702,219","620,306","81,913",10420.0
3,"Albany, GA (MA)","66,756","42,590","24,166","148,922","92,106","56,816",10500.0
4,"Albany-Lebanon, OR (MA)","51,921","34,615","17,306","128,610","84,567","44,043",10540.0
5,"Albany-Schenectady-Troy, NY (MA)","417,490","328,774","88,716","899,262","710,008","189,254",10580.0
...,...,...,...,...,...,...,...,...
386,"Yakima, WA (MA)","90,504","64,399","26,105","256,728","178,600","78,128",49420.0
388,"York-Hanover, PA (MA)","187,187","134,538","52,649","456,438","326,551","129,887",49620.0
389,"Youngstown-Warren-Boardman, OH-PA (MA)","253,211","190,364","62,847","541,243","397,536","143,707",49660.0
390,"Yuba City, CA (MA)","63,957","47,984","15,973","181,208","142,123","39,085",49700.0


,GeoFIPS,GeoName,total_housing,urban_housing,rural_housing,total_pop,urban_pop,rural_pop
0,10180,"Abilene, TX (MA)",74341.0,50514.0,23827.0,176579.0,118138.0,58441.0
1,10420,"Akron, OH (MA)",317145.0,282871.0,34274.0,702219.0,620306.0,81913.0
2,10500,"Albany, GA (MA)",66756.0,42590.0,24166.0,148922.0,92106.0,56816.0
3,10540,"Albany-Lebanon, OR (MA)",51921.0,34615.0,17306.0,128610.0,84567.0,44043.0
4,10580,"Albany-Schenectady-Troy, NY (MA)",417490.0,328774.0,88716.0,899262.0,710008.0,189254.0
...,...,...,...,...,...,...,...,...
379,49420,"Yakima, WA (MA)",90504.0,64399.0,26105.0,256728.0,178600.0,78128.0
380,49620,"York-Hanover, PA (MA)",187187.0,134538.0,52649.0,456438.0,326551.0,129887.0
381,49660,"Youngstown-Warren-Boardman, OH-PA (MA)",253211.0,190364.0,62847.0,541243.0,397536.0,143707.0
382,49700,"Yuba City, CA (MA)",63957.0,47984.0,15973.0,181208.0,142123.0,39085.0


,GeoFIPS,GeoName,urban_housing,rural_housing,urban_pop,rural_pop,rural_pop_prct,rural_housing_prct
0,10180,"Abilene, TX (MA)",50514.0,23827.0,118138.0,58441.0,0.330962,0.320510
1,10420,"Akron, OH (MA)",282871.0,34274.0,620306.0,81913.0,0.116649,0.108070
2,10500,"Albany, GA (MA)",42590.0,24166.0,92106.0,56816.0,0.381515,0.362005
3,10540,"Albany-Lebanon, OR (MA)",34615.0,17306.0,84567.0,44043.0,0.342454,0.333314
4,10580,"Albany-Schenectady-Troy, NY (MA)",328774.0,88716.0,710008.0,189254.0,0.210455,0.212499
...,...,...,...,...,...,...,...,...
379,49420,"Yakima, WA (MA)",64399.0,26105.0,178600.0,78128.0,0.304322,0.288440
380,49620,"York-Hanover, PA (MA)",134538.0,52649.0,326551.0,129887.0,0.284567,0.281264
381,49660,"Youngstown-Warren-Boardman, OH-PA (MA)",190364.0,62847.0,397536.0,143707.0,0.265513,0.248200
382,49700,"Yuba City, CA (MA)",47984.0,15973.0,142123.0,39085.0,0.215691,0.249746


In [ ]:
# dtype_mapping = {'STATE': str, 'COUNTY': str}
# urbanization = pd.read_csv(f"{grandparent_dir}/data/raw/2020_UA_COUNTY.csv", dtype=dtype_mapping)



In [2]:
industry_ts = pd.read_csv(f"{grandparent_dir}/data/raw/industry_time_series_people.csv")

industry_ts["GEO_ID"] = industry_ts["GEO_ID"].str.split("US").str[1]
industry_ts["GEO_ID"] = industry_ts["GEO_ID"].astype("int64")
industry_ts = industry_ts.rename(columns={"GEO_ID": "GeoFIPS"})

common_fips = np.intersect1d(gdp["GeoFIPS"].unique(), industry_ts["GeoFIPS"].unique())


industry_ts = industry_ts[industry_ts["GeoFIPS"].isin(common_fips)]


years = industry_ts['Year'].unique()

for year in years:
    year_df = industry_ts[industry_ts['Year'] == year]
    missing_fips = set(common_fips) - set(year_df['GeoFIPS'])
    
    if missing_fips:
        missing_data = {'Year': [year] * len(missing_fips), 'GeoFIPS': list(missing_fips)}
        
        # Fill all columns from the fourth column (index 3) onward with 0
        for col in industry_ts.columns[2:]:
            missing_data[col] = 0
        
        missing_df = pd.DataFrame(missing_data)
        industry_ts = pd.concat([industry_ts, missing_df], ignore_index=True)

industry_ts = industry_ts.merge(gdp[["GeoFIPS", "GeoName"]], on="GeoFIPS", how="left")


industry_ts = industry_ts[
    [
        "GeoFIPS",
        "GeoName",
        "Year",
        'agriculture_total',
        'mining_total',
        'construction_total',
        'manufacturing_total',
        'wholesale_trade_total',
        'retail_trade_total',
        'transportation_warehousing_total',
        'utilities_total',
        'information_total',
        'finance_insurance_total',
        'real_estate_total',
        'professional_services_total',
        'management_enterprises_total',
        'admin_support_services_total',
        'educational_services_total',
        'healthcare_social_services_total',
        'arts_recreation_total',
        'accommodation_food_services_total',
        'other_services_total',
        'public_administration_total',
    ]
]

industry_ts = industry_ts.sort_values(by=["GeoFIPS", "GeoName", "Year"])

In [5]:

missing_values = industry_ts.isnull().sum()

missing_values

GeoFIPS                              0
GeoName                              0
Year                                 0
agriculture_total                    0
mining_total                         0
construction_total                   0
manufacturing_total                  0
wholesale_trade_total                0
retail_trade_total                   0
transportation_warehousing_total     0
utilities_total                      0
information_total                    0
finance_insurance_total              0
real_estate_total                    0
professional_services_total          0
management_enterprises_total         0
admin_support_services_total         0
educational_services_total           0
healthcare_social_services_total     0
arts_recreation_total                0
accommodation_food_services_total    0
other_services_total                 0
public_administration_total          0
dtype: int64

In [115]:
columns_to_save = industry_ts.columns[industry_ts.columns.get_loc('Year') + 1:]


for column in columns_to_save:

    selected_columns = ['GeoFIPS', 'GeoName', 'Year', column]
    subsetindustry_ts = industry_ts[selected_columns]

    subsetindustry_ts.rename(columns={column: 'Value'}, inplace=True)
    

    subsetindustry_ts_long = subsetindustry_ts.copy()

    file_name_long = f"industry_{column}_long.csv"
    subsetindustry_ts_long.to_csv(f"{grandparent_dir}/data/processed/dump/{file_name_long}", index=False)



    subsetindustry_ts_std_long = standardize_and_scale(subsetindustry_ts)

    file_name_std = f"industry_{column}_std_long.csv"
    subsetindustry_ts_std_long.to_csv(f"{grandparent_dir}/data/processed/dump/{file_name_std}", index=False)


    subsetindustry_ts_wide = subsetindustry_ts.pivot_table(
    index=["GeoFIPS", "GeoName"], columns="Year", values="Value"
    )
    subsetindustry_ts_wide.reset_index(inplace=True)
    subsetindustry_ts_wide.columns.name = None

    
    file_name_wide = f"industry_{column}_wide.csv"
    subsetindustry_ts_wide.to_csv(f"{grandparent_dir}/data/processed/dump/{file_name_wide}", index=False)
    
    

    subsetindustry_ts_std_wide = standardize_and_scale(subsetindustry_ts_wide)

    assert (subsetindustry_ts_std_wide[column] >= -1).all() and subsetindustry_ts_std_wide[column] <= 1).all(), 'STD ERROR'



    file_name_std_wide = f"industry_{column}_std_wide.csv"
    subsetindustry_ts_std_wide.to_csv(f"{grandparent_dir}/data/processed/dump/{file_name_std_wide}", index=False)

    # first standardize then save

/tmp/ipykernel_13900/2843878824.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subsetindustry_ts.rename(columns={column: 'Value'}, inplace=True)


KeyError: 'agriculture_total'

In [ ]:
assert (data.std_wide[feature][column] >= -1).all() and (
                    data.std_wide[feature][column] <= 1
                ).all(), std_error

In [108]:
columns_to_check = industry_ts.columns[3:]

# Check for values outside the range [0, 1,000,000] in these columns
values_outside_range = industry_ts[columns_to_check].apply(lambda x: (x < 0) | (x > 1000000))

# Display rows where values are outside the range
rows_with_values_outside_range = industry_ts[values_outside_range.any(axis=1)]



In [109]:
rows_with_values_outside_range

,GeoFIPS,GeoName,Year,agriculture_total,mining_total,construction_total,manufacturing_total,wholesale_trade_total,retail_trade_total,transportation_warehousing_total,...,real_estate_total,professional_services_total,management_enterprises_total,admin_support_services_total,educational_services_total,healthcare_social_services_total,arts_recreation_total,accommodation_food_services_total,other_services_total,public_administration_total


In [4]:
import pickle

# Replace 'your_file.pkl' with the path to your pickle file
with open(f"{grandparent_dir}/data/raw/exclusions.pkl", 'rb') as file:
    loaded_data = pickle.load(file)

loaded_data


with open(f"{grandparent_dir}/data/raw/exclusions.pkl", 'rb') as file:
    loaded_data = pickle.load(file)

# Save the loaded data to a CSV file
#loaded_data.to_csv('your_output_file.csv', index=False) 

{'transport': array([ 2282, 15901, 22051, 22071, 22075, 22087, 22089, 22095, 22103,
        51901, 51903, 51907, 51911, 51913, 51918, 51919, 51921, 51923,
        51929, 51931, 51933, 51939, 51941, 51942, 51944, 51945, 51947,
        51949, 51951, 51953, 51955, 51958])}

In [6]:

with open(f"{grandparent_dir}/data/raw/exclusions.pkl", 'rb') as file:
    loaded_data = pickle.load(file)


loaded_data

{'transport': array([ 2282, 15901, 22051, 22071, 22075, 22087, 22089, 22095, 22103,
        51901, 51903, 51907, 51911, 51913, 51918, 51919, 51921, 51923,
        51929, 51931, 51933, 51939, 51941, 51942, 51944, 51945, 51947,
        51949, 51951, 51953, 51955, 51958])}

In [12]:
df = pd.DataFrame.from_dict(loaded_data)  # 'orient' specifies that the keys should be used as the index

df


melted_df = pd.melt(df, var_name='dataset', value_name='exclusions')
melted_df['dataset'] = 'transport'
melted_df

,dataset,exclusions
0,transport,2282
1,transport,15901
2,transport,22051
3,transport,22071
4,transport,22075
5,transport,22087
6,transport,22089
7,transport,22095
8,transport,22103
9,transport,51901


In [44]:
industry_ts 

,GeoFIPS,GeoName,Year,agriculture_total,mining_total,construction_total,manufacturing_total,wholesale_trade_total,retail_trade_total,transportation_warehousing_total,...,real_estate_total,professional_services_total,management_enterprises_total,admin_support_services_total,educational_services_total,healthcare_social_services_total,arts_recreation_total,accommodation_food_services_total,other_services_total,public_administration_total
1067,1001,"Autauga, AL",2010,0.006738,0.002711,0.077400,0.141284,0.031141,0.126659,0.032743,...,0.017419,0.045027,0.000246,0.024691,0.068567,0.105748,0.017419,0.069348,0.045520,0.116676
4180,1001,"Autauga, AL",2011,0.010150,0.002320,0.069804,0.139650,0.036290,0.124984,0.042255,...,0.016902,0.043042,0.000000,0.027466,0.075314,0.107834,0.016778,0.064833,0.044285,0.112722
9238,1001,"Autauga, AL",2012,0.010860,0.003482,0.056456,0.135876,0.029720,0.134964,0.044933,...,0.016456,0.043026,0.000000,0.031171,0.072746,0.115233,0.019150,0.058860,0.044684,0.114031
10327,1001,"Autauga, AL",2013,0.010193,0.003843,0.058027,0.128838,0.030163,0.135272,0.048502,...,0.015666,0.046831,0.000000,0.031374,0.063333,0.113757,0.019092,0.056523,0.059114,0.113924
13388,1001,"Autauga, AL",2014,0.011349,0.004391,0.056124,0.132295,0.031976,0.127863,0.040881,...,0.015118,0.049704,0.000000,0.031852,0.063041,0.115934,0.017686,0.058402,0.061550,0.112993
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23555,56045,"Weston, WY",2017,0.053713,0.192545,0.052245,0.061638,0.012328,0.112122,0.061344,...,0.002348,0.011154,0.000000,0.017317,0.125037,0.093044,0.005283,0.056648,0.025829,0.060757
25620,56045,"Weston, WY",2018,0.048934,0.203890,0.065558,0.050816,0.012861,0.094417,0.073714,...,0.001882,0.016311,0.000000,0.018193,0.115747,0.091280,0.008469,0.053011,0.021330,0.057403
28703,56045,"Weston, WY",2019,0.062093,0.188140,0.049674,0.054331,0.012729,0.093760,0.064266,...,0.004346,0.025147,0.000000,0.022043,0.113940,0.081341,0.008072,0.054020,0.049053,0.064887
32913,56045,"Weston, WY",2020,0.046048,0.163918,0.072509,0.069759,0.010309,0.100344,0.048454,...,0.020619,0.023368,0.000000,0.021649,0.079725,0.101375,0.018557,0.029897,0.053952,0.049485
